<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">TradingAgents: Multi-Agent Financial Analysis with Phoenix Tracing</h1>

This notebook demonstrates how to use individual TradingAgents components with Phoenix tracing for observability.

We'll showcase:
- **Market Analyst Agent**: Analyzes technical indicators and price data
- **News Analyst Agent**: Researches current events and market news
- **Bull Researcher Agent**: Builds bullish investment cases
- **Phoenix Tracing**: Full observability into agent execution

## How to Run This Notebook

### Prerequisites
1. **Python 3.10+** (3.13 recommended)
2. **Jupyter Lab** installed
3. **API Keys** set as environment variables

### Installation

```bash
# Create and activate virtual environment
conda create -n tradingagents python=3.13
conda activate tradingagents

# Install dependencies
pip install -r requirements.txt

# Install Jupyter Lab (if not already installed)
pip install jupyterlab
```

### Set API Keys

```bash
export FINNHUB_API_KEY=your_finnhub_key_here
export OPENAI_API_KEY=your_openai_key_here
```

### Launch Jupyter Lab

```bash
# From project root directory
jupyter lab

# Or specify this notebook directly
jupyter lab notebook/trading_agents_demo.ipynb
```

**Note**: Phoenix will be launched from within the notebook - no Docker required!

## 1. Install Dependencies and Import Libraries

In [1]:
import os
import sys
from pathlib import Path
from getpass import getpass
from typing import Annotated

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

import phoenix as px
from phoenix.otel import register
from openinference.instrumentation.langchain import LangChainInstrumentor

/Users/amjadhossain/github.com/amjadjibon/TradingAgents/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Launch Phoenix

Launch Phoenix in the background to collect trace data from TradingAgents.

Phoenix will run locally and provide a UI to inspect all agent interactions, LLM calls, and tool usage.

In [2]:
(session := px.launch_app()).view()

/Users/amjadhossain/.local/share/uv/python/cpython-3.13.7-macos-aarch64-none/lib/python3.13/contextlib.py:148: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
/Users/amjadhossain/.local/share/uv/python/cpython-3.13.7-macos-aarch64-none/lib/python3.13/contextlib.py:148: SAWarning: Skipped unsupported reflection of expression-based index ix_latency
  next(self.gen)


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix
📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/


## 3. Configure API Keys and Instrument LangChain

Set up OpenAI API keys and instrument LangChain to send traces to Phoenix.

In [3]:
# Check OpenAI API key
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

# Check Finnhub API key (optional for this demo with mock data)
if not (finnhub_api_key := os.getenv("FINNHUB_API_KEY")):
    print("⚠️  FINNHUB_API_KEY not set (optional for this demo with mock data)")
else:
    print("✓ Finnhub API key is set")

print("✓ OpenAI API key is set")

# Instrument LangChain to send traces to Phoenix
tracer_provider = register()
LangChainInstrumentor(tracer_provider=tracer_provider).instrument(skip_dep_check=True)

print("\n" + "=" * 60)
print("✓ LangChain Instrumentation Enabled")
print("=" * 60)
print(f"View traces at: {session.url}")
print("=" * 60)

✓ Finnhub API key is set
✓ OpenAI API key is set
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.


✓ LangChain Instrumentation Enabled
View traces at: http://localhost:6006/


## 4. Define Mock Trading Tools

Create simplified versions of TradingAgents tools for demonstration.

In [4]:
@tool
def get_market_data(
    ticker: Annotated[str, "Stock ticker symbol (e.g., 'AAPL', 'NVDA')"],
    date: Annotated[str, "Date in YYYY-MM-DD format"]
) -> str:
    """
    Retrieve market data and technical indicators for a stock.
    Returns price data, moving averages, RSI, MACD, and volume info.
    """
    # Mock data for demonstration
    return f"""Market Data for {ticker} on {date}:
    
Price Data:
- Current Price: $450.25
- Open: $445.00
- High: $455.50
- Low: $444.00
- Volume: 45.2M shares

Technical Indicators:
- RSI (14): 58.3 (Neutral - neither overbought nor oversold)
- MACD: 2.45 (Bullish - above signal line)
- 50-day SMA: $435.80 (Price above - bullish signal)
- 200-day SMA: $410.50 (Price above - long-term uptrend)
- Bollinger Bands: Price near middle band, moderate volatility

Volume Analysis:
- Average Volume (30d): 42.1M
- Volume Trend: Above average (+7.4%)
"""

@tool
def get_news_sentiment(
    ticker: Annotated[str, "Stock ticker symbol"],
    date: Annotated[str, "Date in YYYY-MM-DD format"]
) -> str:
    """
    Retrieve recent news and sentiment analysis for a stock.
    Returns news headlines, sentiment scores, and market impact.
    """
    # Mock data for demonstration
    return f"""News & Sentiment for {ticker} on {date}:

Recent Headlines (Past 7 days):
1. "{ticker} reports strong Q4 earnings, beats analyst estimates" - Positive (Score: 0.82)
2. "New AI chip announcement drives {ticker} stock higher" - Positive (Score: 0.75)
3. "Industry analysts upgrade {ticker} price target to $500" - Positive (Score: 0.68)
4. "Supply chain concerns weigh on tech sector including {ticker}" - Negative (Score: -0.45)

Sentiment Summary:
- Overall Sentiment: Positive (0.62 average)
- Positive News: 75%
- Neutral News: 15%
- Negative News: 10%

Social Media Sentiment:
- Twitter Mentions: 15,400 (↑25% vs. previous week)
- Reddit r/stocks sentiment: Mostly bullish
- Analyst Ratings: 12 Buy, 3 Hold, 1 Sell
"""

@tool
def get_fundamentals(
    ticker: Annotated[str, "Stock ticker symbol"]
) -> str:
    """
    Retrieve fundamental analysis data for a stock.
    Returns financial metrics, valuation ratios, and growth indicators.
    """
    # Mock data for demonstration
    return f"""Fundamental Analysis for {ticker}:

Financial Metrics:
- Market Cap: $1.12T
- P/E Ratio: 28.5 (slightly above sector average of 25.2)
- PEG Ratio: 1.8 (reasonable growth valuation)
- Revenue (TTM): $96.3B (↑15.2% YoY)
- Net Income (TTM): $42.1B (↑22.8% YoY)
- Operating Margin: 43.7% (industry-leading)

Balance Sheet:
- Cash & Equivalents: $28.5B
- Total Debt: $15.2B
- Debt-to-Equity: 0.35 (conservative leverage)
- Current Ratio: 2.1 (healthy liquidity)

Growth Indicators:
- Revenue Growth (3Y CAGR): 18.5%
- EPS Growth (3Y CAGR): 24.2%
- Analyst Estimates (Next Year): +16% revenue growth

Insider Activity:
- Recent Insider Buying: $2.3M (3 transactions)
- Institutional Ownership: 68.5%
"""

print("✓ Trading tools defined")

✓ Trading tools defined


## 5. Create Market Analyst Agent

This agent analyzes technical indicators and market trends.

In [5]:
def create_market_analyst(llm, tools):
    """Create a market analyst agent that analyzes technical indicators."""
    
    system_message = """You are a Market Analyst specializing in technical analysis.
    
Your role is to:
- Analyze price movements, volume trends, and technical indicators
- Identify support and resistance levels
- Assess momentum using RSI, MACD, and moving averages
- Provide clear, actionable insights for traders

Guidelines:
- Be specific with numbers and percentages
- Explain what indicators suggest about market momentum
- Highlight any conflicting signals
- Present findings in a structured, easy-to-read format
"""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("human", "Analyze the market data for {ticker} on {date}. Use the available tools to gather data and provide a comprehensive technical analysis."),
        MessagesPlaceholder(variable_name="messages"),
    ])
    
    chain = prompt | llm.bind_tools(tools)
    
    def analyst_node(ticker: str, date: str, messages: list = None):
        if messages is None:
            messages = []
        
        result = chain.invoke({"ticker": ticker, "date": date, "messages": messages})
        return result
    
    return analyst_node

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create market analyst
market_analyst = create_market_analyst(llm, [get_market_data])

print("✓ Market Analyst agent created")

✓ Market Analyst agent created


## 6. Run Market Analyst

Execute the market analyst agent and view traces in Phoenix.

In [6]:
ticker = "NVDA"
date = "2024-05-10"

print(f"Analyzing {ticker} on {date}...\n")

# First call - agent will request tool use
result = market_analyst(ticker, date)

print("=" * 60)
print("AGENT RESPONSE (Tool Calls)")
print("=" * 60)
print(f"Tool Calls: {len(result.tool_calls)}")
for tool_call in result.tool_calls:
    print(f"  - {tool_call['name']}: {tool_call['args']}")

# Execute tools and provide results back to agent
messages = [result]
for tool_call in result.tool_calls:
    if tool_call['name'] == 'get_market_data':
        tool_result = get_market_data.invoke(tool_call['args'])
        # Use ToolMessage instead of AIMessage
        messages.append(ToolMessage(
            content=tool_result,
            tool_call_id=tool_call['id']
        ))

# Second call - agent will provide analysis
final_result = market_analyst(ticker, date, messages)

print("\n" + "=" * 60)
print("MARKET ANALYST REPORT")
print("=" * 60)
print(final_result.content)
print("=" * 60)

print(f"\n📊 View traces at: {session.url}")

Analyzing NVDA on 2024-05-10...

AGENT RESPONSE (Tool Calls)
Tool Calls: 1
  - get_market_data: {'ticker': 'NVDA', 'date': '2024-05-10'}

MARKET ANALYST REPORT
### Technical Analysis of NVDA (NVIDIA Corporation) on May 10, 2024

#### Price Overview
- **Current Price**: $450.25
- **Open**: $445.00
- **High**: $455.50
- **Low**: $444.00
- **Volume**: 45.2M shares

#### Key Technical Indicators
1. **Relative Strength Index (RSI)**: 
   - **Value**: 58.3
   - **Interpretation**: The RSI is in the neutral zone (40-60), indicating that the stock is neither overbought nor oversold. This suggests that there is potential for further upward movement without immediate selling pressure.

2. **Moving Average Convergence Divergence (MACD)**:
   - **Value**: 2.45
   - **Interpretation**: The MACD is above the signal line, indicating a bullish momentum. This suggests that the stock may continue to experience upward price movement.

3. **Simple Moving Averages (SMA)**:
   - **50-day SMA**: $435.80
    

## 7. Create News Analyst Agent

This agent researches news and sentiment to assess market mood.

In [8]:
def create_news_analyst(llm, tools):
    """Create a news analyst agent that researches current events."""
    
    system_message = """You are a News Analyst specializing in market sentiment analysis.
    
Your role is to:
- Analyze recent news headlines and their potential market impact
- Assess overall sentiment (bullish, bearish, or neutral)
- Identify key catalysts or risk factors
- Evaluate social media and analyst sentiment

Guidelines:
- Distinguish between noise and meaningful news
- Consider the credibility and timing of news sources
- Look for contradictions between different sentiment sources
- Provide balanced analysis even when sentiment is strongly directional
"""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("human", "Research news and sentiment for {ticker} on {date}. Use the available tools to gather information and provide a comprehensive sentiment analysis."),
        MessagesPlaceholder(variable_name="messages"),
    ])
    
    chain = prompt | llm.bind_tools(tools)
    
    def analyst_node(ticker: str, date: str, messages: list = None):
        if messages is None:
            messages = []
        
        result = chain.invoke({"ticker": ticker, "date": date, "messages": messages})
        return result
    
    return analyst_node

# Create news analyst
news_analyst = create_news_analyst(llm, [get_news_sentiment])

print("✓ News Analyst agent created")

✓ News Analyst agent created


## 8. Run News Analyst

Execute the news analyst agent.

In [10]:
print(f"Researching news for {ticker} on {date}...\n")

# First call - agent will request tool use
result = news_analyst(ticker, date)

# Execute tools
messages = [result]
for tool_call in result.tool_calls:
    if tool_call['name'] == 'get_news_sentiment':
        tool_result = get_news_sentiment.invoke(tool_call['args'])
        # Use ToolMessage instead of AIMessage
        messages.append(ToolMessage(
            content=tool_result,
            tool_call_id=tool_call['id']
        ))

# Second call - agent will provide analysis
final_result = news_analyst(ticker, date, messages)

print("=" * 60)
print("NEWS ANALYST REPORT")
print("=" * 60)
print(final_result.content)
print("=" * 60)

print(f"\n📊 View traces at: {session.url}")

Researching news for NVDA on 2024-05-10...

NEWS ANALYST REPORT
### Comprehensive Sentiment Analysis for NVIDIA (NVDA) on May 10, 2024

#### Recent News Headlines
1. **"NVDA reports strong Q4 earnings, beats analyst estimates"** - **Positive** (Score: 0.82)
2. **"New AI chip announcement drives NVDA stock higher"** - **Positive** (Score: 0.75)
3. **"Industry analysts upgrade NVDA price target to $500"** - **Positive** (Score: 0.68)
4. **"Supply chain concerns weigh on tech sector including NVDA"** - **Negative** (Score: -0.45)

#### Sentiment Summary
- **Overall Sentiment**: **Positive** (Average Score: 0.62)
- **Positive News**: 75%
- **Neutral News**: 15%
- **Negative News**: 10%

#### Key Catalysts
- **Strong Earnings Report**: NVDA's Q4 earnings exceeded analyst expectations, which typically boosts investor confidence and can lead to a rise in stock price.
- **AI Chip Announcement**: The introduction of a new AI chip is likely to enhance NVDA's competitive position in the rapidly g

## 9. Create Bull Researcher Agent

This agent builds a bullish investment case based on analyst reports.

In [11]:
def create_bull_researcher(llm):
    """Create a bull researcher agent that advocates for investment."""
    
    def bull_node(ticker: str, market_report: str, news_report: str, fundamentals: str):
        prompt = f"""You are a Bull Analyst advocating for investing in {ticker}.

Your task is to build a strong, evidence-based case emphasizing:
- Growth potential and market opportunities
- Competitive advantages and market positioning
- Positive technical indicators and momentum
- Favorable sentiment and news catalysts
- Strong fundamentals and financial health

Resources available:

Market Analysis:
{market_report}

News & Sentiment:
{news_report}

Fundamentals:
{fundamentals}

Guidelines:
- Use specific data points and metrics to support your arguments
- Address potential concerns proactively
- Present a compelling but honest bull case
- Organize your argument clearly with key points highlighted
- Conclude with a conviction level (Strong Buy / Buy / Moderate Buy)

Provide your bull analysis:
"""
        
        response = llm.invoke(prompt)
        return response.content
    
    return bull_node

# Create bull researcher
bull_researcher = create_bull_researcher(llm)

print("✓ Bull Researcher agent created")

✓ Bull Researcher agent created


## 10. Run Complete Analysis Pipeline

Execute all agents in sequence to build a complete investment case.

## 11. Multi-Ticker Analysis

Run analysis on multiple tickers to compare opportunities.

In [13]:
tickers_to_analyze = ["AAPL", "MSFT", "GOOGL"]
date = "2024-05-10"
results = {}

print(f"Analyzing {len(tickers_to_analyze)} tickers: {', '.join(tickers_to_analyze)}\n")

for ticker in tickers_to_analyze:
    print(f"\nAnalyzing {ticker}...")
    
    try:
        # Market analysis
        market_result = market_analyst(ticker, date)
        market_messages = [market_result]
        for tool_call in market_result.tool_calls:
            tool_result = get_market_data.invoke(tool_call['args'])
            market_messages.append(ToolMessage(
                content=tool_result,
                tool_call_id=tool_call['id']
            ))
        market_analysis = market_analyst(ticker, date, market_messages)
        
        # News analysis
        news_result = news_analyst(ticker, date)
        news_messages = [news_result]
        for tool_call in news_result.tool_calls:
            tool_result = get_news_sentiment.invoke(tool_call['args'])
            news_messages.append(ToolMessage(
                content=tool_result,
                tool_call_id=tool_call['id']
            ))
        news_analysis = news_analyst(ticker, date, news_messages)
        
        # Fundamentals
        fundamentals = get_fundamentals.invoke({"ticker": ticker})
        
        # Bull case
        bull_case = bull_researcher(ticker, market_analysis.content, news_analysis.content, fundamentals)
        
        results[ticker] = {
            "market": market_analysis.content,
            "news": news_analysis.content,
            "bull_case": bull_case
        }
        
        print(f"✓ {ticker} analysis complete")
        
    except Exception as e:
        print(f"❌ {ticker}: Error - {e}")
        results[ticker] = {"error": str(e)}

# Display summary
print("\n" + "=" * 80)
print("MULTI-TICKER ANALYSIS SUMMARY")
print("=" * 80)

for ticker, result in results.items():
    print(f"\n{ticker}:")
    if "error" in result:
        print(f"  ❌ Error: {result['error']}")
    else:
        # Extract first line of bull case as summary
        summary = result['bull_case'].split('\n')[0][:100]
        print(f"  {summary}...")

print("\n" + "=" * 80)
print(f"\n📊 View all traces at: {session.url}")

Analyzing 3 tickers: AAPL, MSFT, GOOGL


Analyzing AAPL...
✓ AAPL analysis complete

Analyzing MSFT...
✓ MSFT analysis complete

Analyzing GOOGL...
✓ GOOGL analysis complete

MULTI-TICKER ANALYSIS SUMMARY

AAPL:
  ### Bull Case for Investing in AAPL...

MSFT:
  ### Bullish Analysis for Microsoft Corporation (MSFT)...

GOOGL:
  ### Bull Case for Investing in GOOGL...


📊 View all traces at: http://localhost:6006/


## 12. Explore Phoenix UI

After running the analyses, explore the Phoenix UI to see:

1. **Traces Tab**: View execution traces for each agent
   - Agent execution flow
   - LLM prompts and responses
   - Tool calls with arguments and results
   - Latency for each step

2. **Token Usage**: Monitor API costs
   - Total tokens per trace
   - Breakdown by agent and model

3. **Spans**: Drill down into specific operations
   - Market data retrieval
   - News sentiment analysis
   - Bull case generation

**Open Phoenix UI**: Access via `session.url` or http://localhost:6006/

## 13. Cleanup

Uninstrument LangChain and stop Phoenix.

In [14]:
# Uninstrument LangChain
try:
    LangChainInstrumentor().uninstrument()
    print("✓ LangChain instrumentation cleaned up")
except Exception as e:
    print(f"Cleanup warning: {e}")

print("\nPhoenix will continue running in the background.")
print(f"Access at: {session.url}")

✓ LangChain instrumentation cleaned up

Phoenix will continue running in the background.
Access at: http://localhost:6006/
